# Análisis ponderado de acciones de S&P a mediano plazo
## Este programa utiliza Yahoo Finance para analizar y ponderar todas las acciones de S&P 500 en el último año para luego visualizar las mejores 10 inversiones a mediano plazo
### El análisis en cada parte se hace con la acción AAPL, pero luego se hace con todas las demás
### El programa está diseñado para trabajar en conjunto con una apreciación personal de las inversiones, no para invertir ciegamente en el top de acciones que retorna

### Librerías necesarias

In [ ]:
!pip install pandas_ta

import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import time
from ipywidgets import interact, widgets
from datetime import datetime

### Descarga de datos con Yahoo Finance

In [269]:
def get_data(ticker, start_date='2024-01-01'):
    # Obtener la fecha actual
    end_date = datetime.today().strftime('%Y-%m-%d')

    # Obtener datos históricos
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    return data

# Ejemplo de acción
ticker = 'AAPL'
data = get_data(ticker)
data

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2024-01-02,184.532074,187.315366,182.792518,186.033057,82488700
2024-01-03,183.150375,184.770652,182.335262,183.120556,58414500
2024-01-04,180.824356,181.997307,179.800504,181.062914,71983600
2024-01-05,180.098709,181.669281,179.094742,180.903888,62303300
2024-01-08,184.452560,184.492330,180.416793,181.003268,59144500
...,...,...,...,...,...
2025-03-03,238.029999,244.029999,236.110001,241.789993,47184000
2025-03-04,235.929993,240.070007,234.679993,237.710007,53798100


### Cálculo de indicadores técnicos y muestra

In [270]:
def calculate_indicators(data, ticker):
    # Calcular MACD (diferencia entre EMA 12 y EMA 26)
    data['MACD'], data['MACD_signal'], _ = ta.macd(data[('Close', ticker)])

    # Calcular RSI (Índice de Fuerza Relativa)
    data['RSI'] = ta.rsi(data[('Close', ticker)])

    # Calcular ADX (Average Directional Index, mide la fuerza de la tendencia)
    adx_result = ta.adx(data[('High', ticker)], data[('Low', ticker)], data[('Close', ticker)])
    data[('ADX', ticker)] = adx_result['ADX_14']  # Asignar ADX
    data[('ADX_pos', ticker)] = adx_result['DMP_14']  # Asignar ADX Positive (DMP_14)
    data[('ADX_neg', ticker)] = adx_result['DMN_14']  # Asignar ADX Negative (DMN_14)

    # Calcular Bandas de Bollinger (Medimos si el precio está en zona de sobrecompra/sobreventa)
    bbands_result = ta.bbands(data[('Close', ticker)])
    data[('Upper_Band', ticker)] = bbands_result['BBU_5_2.0']  # Asignar Upper Band
    data[('Lower_Band', ticker)] = bbands_result['BBL_5_2.0']  # Asignar Lower Band

    return data

# Ejemplo de indicadores
data_with_indicators = calculate_indicators(data, ticker)
data_with_indicators[['MACD', 'RSI', 'ADX', 'Upper_Band', 'Lower_Band']].tail()

Price,MACD,RSI,ADX,Upper_Band,Lower_Band
Ticker,,,AAPL,AAPL,AAPL
Date,,,,,
2025-03-03,MACD_12_26_9,48.447005,20.376333,247.846277,233.981719
2025-03-04,MACD_12_26_9,45.885176,19.598156,242.953143,234.430854
2025-03-05,MACD_12_26_9,45.649990,19.759878,242.182507,233.353491
2025-03-06,MACD_12_26_9,45.112604,19.612410,242.216187,232.531811
2025-03-07,MACD_12_26_9,50.801949,18.720497,239.746463,233.893539


### Obtención de métricas fundamentales

In [271]:
def get_fundamentals(ticker):
    stock = yf.Ticker(ticker)
    time.sleep(1)
    # Obtener el reporte financiero
    try:
        fundamentals = stock.info
        pe_ratio = fundamentals.get('trailingPE', None)  # P/E ratio
        pb_ratio = fundamentals.get('priceToBook', None)  # P/B ratio
        roe = fundamentals.get('returnOnEquity', None)  # ROE

        return pe_ratio, pb_ratio, roe
    except Exception as e:
        print(f"Error obteniendo datos fundamentales para {ticker}: {e}")
        return None, None, None

# Ejemplo de métricas fundamentales
get_fundamentals(ticker)

(35.716354, 50.781925, 1.3652)

### Cálculo el beta de la acción (para la volatilidad)

In [272]:
def calculate_beta(ticker, benchmark_ticker='^GSPC', start_date='2024-01-01'):
    # Obtener los datos históricos de la acción y el índice de referencia
    data = yf.download([ticker, benchmark_ticker], start=start_date, progress=False)

    # Acceder a las columnas 'Close' para la acción y el índice
    data_close = data['Close']

    # Calcular los rendimientos diarios de la acción y el índice
    returns = data_close.pct_change().dropna()

    # Calcular la covarianza entre la acción y el índice
    covariance = np.cov(returns[ticker], returns[benchmark_ticker])[0][1]

    # Calcular la varianza del índice de referencia
    variance = np.var(returns[benchmark_ticker])

    # Calcular el beta
    beta = covariance / variance

    return beta

# Ejemplo del beta
calculate_beta(ticker)

0.9422256315915455

## Cálculo de la tendencia reciente de la acción (último año)

In [273]:
def calculate_recent_trend(data, months=12):
    # Calcular el número de días en el periodo deseado (aproximadamente 30 días por mes)
    days_in_period = months * 30

    # Verificar que haya suficientes datos
    if len(data) >= days_in_period:
        # Calcular la variación porcentual en el periodo de 'months' meses
        recent_change = (data['Close'].iloc[-1] / data['Close'].iloc[-days_in_period]) - 1
    else:
        # Si no hay suficientes datos, usar el primer y el último día disponible
        recent_change = (data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1

    return recent_change.mean()

### Definición de pesos para cada métrica y análisis ponderado



In [274]:
def calculate_weighted_score(data, ticker):
    # Asignar pesos a cada indicador

    # weight_macd = 0.25
    # weight_rsi = 0.2
    # weight_adx = 0.2
    # weight_bbands = 0.2
    # weight_pe = 0.1
    # weight_pb = 0.1
    # weight_roe = 0.1
    # weight_volatilidad = 0.1
    # weight_sharpe = 0.1
    # weight_beta = 0.1
    # weight_drawdown = 0.1
    # Esta normalización no favorece tanto a desplomes recientes, por lo tanto, le doy más importancia a la volatilidad de la acción

    weight_macd = 0.1
    weight_rsi = 0.1
    weight_adx = 0.1
    weight_bbands = 0.1
    weight_pe = 0.05
    weight_pb = 0.05
    weight_roe = 0.05
    weight_volatilidad = 0.25  # Aumentamos el peso de volatilidad
    weight_sharpe = 0.05
    weight_beta = 0.1
    weight_drawdown = 0.2  # Aumentamos el peso de drawdown

    # Normalizar y combinar los indicadores (simplificación)
    macd_score = (data['MACD'].dropna().align(data['MACD_signal'].dropna(), join='inner')[0] >
              data['MACD_signal'].dropna().align(data['MACD'].dropna(), join='inner')[0]).mean()

    rsi_score = (data['RSI'] < 30).mean()  # Sobreventa

    adx_score = (data['ADX'] > 25).mean()  # Tendencia fuerte

    bbands_score = ((data['Close'] > data['Upper_Band']).mean())  # Precio en la banda superior

    # Obtener datos fundamentales
    pe_ratio, pb_ratio, roe = get_fundamentals(ticker)

    # Calcular puntajes fundamentales
    pe_score = 1 if pe_ratio and pe_ratio < 15 else 0  # Bajo P/E es bueno
    pb_score = 1 if pb_ratio and pb_ratio < 1 else 0   # Bajo P/B es bueno
    roe_score = 1 if roe and roe > 0.15 else 0  # ROE positivo y alto es bueno

    # Calcular indicadores de riesgo
    volatilidad = data['Close'].pct_change().std()  # Desviación estándar (volatilidad)
    sharpe_ratio = (data['Close'].pct_change().mean() - 0.02) / data['Close'].pct_change().std()  # Sharpe ratio
    beta = calculate_beta(ticker)  # Beta comparado con el S&P 500
    drawdown = (data['Close'].pct_change().cumprod() / data['Close'].pct_change().cumprod().cummax()).min()  # Drawdown

    # Normalizar puntajes de riesgo
    volatilidad_score = 1 - volatilidad  # Menos volatilidad es mejor
    sharpe_score = sharpe_ratio  # Mayor Sharpe es mejor
    beta_score = 1 / (1 + beta)  # Menor beta es menos riesgoso
    drawdown_score = 1 - drawdown  # Menor drawdown es mejor

    # Calcular el puntaje ponderado incluyendo todos los indicadores y el riesgo
    score = (weight_macd * macd_score +
             weight_rsi * rsi_score +
             weight_adx * adx_score +
             weight_bbands * bbands_score +
             weight_pe * pe_score +
             weight_pb * pb_score +
             weight_roe * roe_score +
             weight_volatilidad * volatilidad_score +
             weight_sharpe * sharpe_score +
             weight_beta * beta_score +
             weight_drawdown * drawdown_score)

    # Filtro adicional: penalización por caídas y bonificación por subidas recientes
    recent_trend = calculate_recent_trend(data)

    if recent_trend < -0.05:  # Penalizar si la caída es mayor al 5%
        score *= 0.9
    elif recent_trend < -0.40:  # Penalizar más fuertemente si la caída es mayor al 40%
      score *= 0.7
    elif recent_trend > 0.05:  # Bonificar si el crecimiento es mayor al 5%
      score *= 1.1

    return score

# Ejemplo de puntaje ponderado
score = calculate_weighted_score(data_with_indicators, ticker)
score

,0
Ticker,
AAPL,0.581262


# Función de análisis de tickers

In [275]:
def analyze_tickers(tickers):
    results = []

    for ticker in tickers:
        data = get_data(ticker)
        if data is None or data.empty:
            continue  # Si no se pueden obtener datos, se omite este ticker
        data_with_indicators = calculate_indicators(data, ticker)

        # Calcular el puntaje ponderado
        score = calculate_weighted_score(data_with_indicators, ticker)
        results.append({'Ticker': ticker, 'Puntaje': float(score.iloc[0])})

    # Crear un DataFrame con los resultados
    results_df = pd.DataFrame(results, columns=['Ticker', 'Puntaje'])
    return results_df

### Obtener tickers del S&P 500 desde Wikipedia

In [276]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)

# La primera tabla contiene la lista de tickers
sp500_tickers = tables[0]['Symbol'].tolist()

# Análisis de todas las acciones del S&P 500

In [ ]:
sp500_analysis = analyze_tickers(sp500_tickers)

# Obtener el top ponderado de acciones

In [279]:
sp500_analysis['Puntaje'] = pd.to_numeric(sp500_analysis['Puntaje'], errors='coerce')
sp500_analysis_sorted = sp500_analysis.sort_values(by='Puntaje', ascending=False)
sp500_analysis_sorted[:30]

,Ticker,Puntaje
223,HCA,0.709392
88,CBOE,0.702981
463,UHS,0.700537
21,MO,0.687992
201,FOX,0.685531
267,K,0.684439
141,DAL,0.680361
200,FOXA,0.677973
258,IRM,0.674150
168,EOG,0.673673
